In [1]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools

In [2]:
random.seed(1356) # set seed so that each play is the same

In [3]:
def hex_add(a, b):
    return Hex[a[0] + b[0], a[1] + b[1], a[2] + b[2]]

def hex_subtract(a, b):
    return Hex[a[0] - b[0], a[1] - b[1], a[2] - b[2]]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_directions[direction])

def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex[0]) + abs(hex[1]) + abs(hex[2])) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

In [4]:
hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

In [5]:
random.seed(1356)
side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 1 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

#Game ends if number of free tiles < 4
total_nr_stones = blackstones + whitestones <4
side

[14024079292171647447, 15846705576740111]

In [6]:
def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

board = initializeBoard()
board, len(board)

({(-1, 0, 1): '',
  (-1, 1, 0): '',
  (0, -1, 1): '',
  (0, 0, 0): '',
  (0, 1, -1): '',
  (1, -1, 0): '',
  (1, 0, -1): ''},
 7)

In [7]:
def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

# wzobristboard

In [8]:
def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard


# bzobristboard

In [9]:
def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex


# zboard

In [11]:
def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

In [12]:
board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces


boardToZobrist(board, zboard, wzobristboard, bzobristboard)

14024079292171647447

In [13]:
def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

In [14]:
# Form groups
# score = find_scores(board, 'w')
def findGroups(score, board):
    new_list = []
    group = {}
    group_state = {}
    group_count = 0

    for tile in score:
        counter = 0
        group[group_count] = 0
        if not tile in new_list:
            for direction in hex_directions:
                new_tile = tuple(map(sum, zip(tile, direction)))
                if new_tile in score:
                    new_list += [new_tile] # Already seen in group

                if score[tile] == board[direction]:
                    group[group_count] += counter
                    group_state.setdefault(tile, []).append(new_tile)

                counter += 1
        group_count += 1
    return group_state

In [15]:
# Check if lists can be grouped by at least 1 common member.
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [16]:
# Count total of each group
# group = findGroups(score, board)

def totalgroups(group):
    count_list = []

    for tile in group:
        for tile2 in group:
            if tile != tile2:
                intersect = list(Intersection(group[tile], group[tile2]))

                if len(intersect) != 0:
                    count_list += [len(list(set(group[tile] + group[tile2])))]
    return count_list


# print(totalgroups(group))

In [183]:
board = {
                 (-1, 0, 1): 'w',
                 (-1, 1, 0): 'b',
                 (0, -1, 1): 'b',
                 (0, 0, 0): '',
                 (0, 1, -1): 'b',
  
                 (1, -1, 0): 'b',
                 (1, 0, -1): 'w'}

In [17]:
score = find_scores(board, 'b') #score of player1
score2 = find_scores(board, 'w') #score of player2

group = findGroups(score, board)
group2 = findGroups(score2, board)

count_list = totalgroups(group)
count_list2 = totalgroups(group2)

print(totalgroups(group), totalgroups(group2))

[] []


In [18]:
# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

result(count_list, count_list2)

(0, 0)

In [19]:
# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [20]:
# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
        
    return board

In [21]:
def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

In [22]:
def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1

In [23]:
#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

In [29]:
#Creating entire the game tree for the play
from copy import deepcopy
import time

def buildGameTree(board, depth):
    depth += 1
    gametree = {}
    tempscore = 1
    
    zboard = initializeBoardZobrist() # initial board, without any pieces
    wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
    bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces
    zidentity0 = boardToZobrist(board, zboard, wzobristboard, bzobristboard) # current root/parent

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    for i in range(len(all_cells)):
        board2 = deepcopy(board) # make sure that you copy the board from the parent!!

        #Only make a move when a move is valid
        valid = False
        
        count = 0
        while not valid:
            if count < 1:
                valid = True
            cell = all_cells[count]
            x, y, _ = cell[0]
            x2, y2, _ = cell[1]
            
            #Make two valid moves
            if validMove(board2, x, y):
                if validMove(board2, x2, y2):
                    makeMove(board2, [x, y], 'b')

                    makeMove(board2, [x2, y2], 'w')
                    valid = True
            
                    # Create a zobrist key for the new board/childnode
                    zidentity = boardToZobrist(board2, zboard, wzobristboard, bzobristboard)

                    # If the board is full, tell the score
                    if fullBoard(board2):
                        
                        score = find_scores(board2, 'b') #score of player1
                        score2 = find_scores(board2, 'w') #score of player2

                        group = findGroups(score, board2)
                        group2 = findGroups(score2, board2)

                        count_list = totalgroups(group)
                        count_list2 = totalgroups(group2)
                        
                        player_one_total, player_two_total = result(count_list, count_list2)
                        
                        # Did player 1 win, lose or draw?
                        game_state = (('l', 'w')[player_one_total < player_two_total ], 'd')[player_one_total == player_two_total]

                        #Store the final game state
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {game_state}])
                        
                    else:
                        gametree.setdefault(zidentity0, []).append([zidentity, board2, depth, tempscore, {}])#childTrees(board2, depth)]) # store full board + temporary score + horizon effect?
            
            count += 1
#             break
        all_cells = all_cells[count:]
    return gametree

start = time.time()
gametree = buildGameTree(board, 0)#13)
end = time.time()
print('Time: ', end - start)
gametree

Time:  0.006253957748413086


{14024079292171647447: [[765384773236229749,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): 'w',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [4743683394420103188,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): 'w',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [13524441802323967158,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): 'w',
    (0, 1, -1): '',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [5985562699070635757,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): 'w',
    (1, -1, 0): '',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [1850940307804494616,
   {(-1, 0, 1): 'b',
    (-1, 1, 0): '',
    (0, -1, 1): '',
    (0, 0, 0): '',
    (0, 1, -1): '',
    (1, -1, 0): 'w',
    (1, 0, -1): ''},
   1,
   1,
   {}],
  [15004330490200760220,
   {(-1, 0, 1): 'b',
  

In [30]:
#Create rest of the game tree
c = 1
threshold = 10#round(len(g)/2)
def walkaroundTree(d, c):
    for n in d:
        
        if d[n] != {} and type(d[n]) == dict:
            walkaroundTree(d[n], c+1)
        elif type(d[n]) == list and len(d[n]) != 0:
            for node in d[n]:
                if node[4] == {}:
                    node[4] = buildGameTree(node[1], c)
#                     if c >= threshold :
#                         return True
#                     else:
                    walkaroundTree(node[4], c+1)
    
    return d


start = time.time()
walkaroundTree(gametree, c)
end = time.time()
print('Time: ', end - start)
gametree[14024079292171647447][0]

Time:  3.042931318283081


[765384773236229749,
 {(-1, 0, 1): 'b',
  (-1, 1, 0): 'w',
  (0, -1, 1): '',
  (0, 0, 0): '',
  (0, 1, -1): '',
  (1, -1, 0): '',
  (1, 0, -1): ''},
 1,
 1,
 {765384773236229749: [[3585104155712041760,
    {(-1, 0, 1): 'b',
     (-1, 1, 0): 'w',
     (0, -1, 1): 'b',
     (0, 0, 0): 'w',
     (0, 1, -1): '',
     (1, -1, 0): '',
     (1, 0, -1): ''},
    2,
    1,
    {3585104155712041760: [[15667646708959168288,
       {(-1, 0, 1): 'b',
        (-1, 1, 0): 'w',
        (0, -1, 1): 'b',
        (0, 0, 0): 'w',
        (0, 1, -1): 'b',
        (1, -1, 0): 'w',
        (1, 0, -1): ''},
       3,
       1,
       {'l'}],
      [1223680409285247908,
       {(-1, 0, 1): 'b',
        (-1, 1, 0): 'w',
        (0, -1, 1): 'b',
        (0, 0, 0): 'w',
        (0, 1, -1): 'b',
        (1, -1, 0): '',
        (1, 0, -1): 'w'},
       3,
       1,
       {'l'}],
      [8906344357997116477,
       {(-1, 0, 1): 'b',
        (-1, 1, 0): 'w',
        (0, -1, 1): 'b',
        (0, 0, 0): 'w',
        (0

In [33]:
def mcts(gametree, numsamples, wincountperinstance, tp_table, previous):
    
    
    for i in range(numsamples):
        
        if gametree != {}:
            rnd_keys = random.choice(list(gametree))
            nodes = gametree[rnd_keys]
            rnd = random.randint(0, len(nodes) - 1)
            instance = nodes[rnd]
            
            if instance[4] != {}:
                if type(instance[4]) != set:
                    instance[3] += 1
                    mcts(instance[4], numsamples, wincountperinstance, tp_table, instance[0])
                else:
                    gamestate = list(instance[4])
                    if instance[4] == {'w'}:
                        instance[3] = 999
                        wincountperinstance.setdefault(instance[0], []).append(999)
                    if instance[4] == {'l'}:
                        instance[3] = -999
                        wincountperinstance.setdefault(instance[0], []).append(-999)
                    if instance[4] == {'d'}:
                        instance[3] = -499
                        wincountperinstance.setdefault(instance[0], []).append(-499)
            
#             [depth, previous, next, score]
            tp_table.setdefault(instance[0], []).append(instance[2])
            tp_table.setdefault(instance[0], []).append(previous)
            tp_table.setdefault(instance[0], []).append(instance[3])
            
#             break

    return gametree, wincountperinstance, tp_table

# mcts_pl = mcts(gametree[14024079292171647447], 2)

for i in range(100):
    start = time.time()
    mcts_pl = mcts(gametree, 100, {}, {}, list(gametree)[0])
    end = time.time()
    print('Time: ', end - start)
# mcts_pl

Time:  8.7688889503479
Time:  10.442008018493652
Time:  9.159588098526001
Time:  8.598006010055542
Time:  9.545894145965576
Time:  8.624241828918457
Time:  11.040211915969849
Time:  8.874428033828735
Time:  8.989383935928345
Time:  8.841494083404541
Time:  9.729753971099854
Time:  12.827832221984863
Time:  9.0362389087677
Time:  8.602140188217163
Time:  12.645493984222412
Time:  10.557467937469482
Time:  9.969150066375732
Time:  9.798149824142456
Time:  14.003636121749878
Time:  10.999728918075562
Time:  9.72112512588501
Time:  9.677414894104004
Time:  8.83589792251587
Time:  8.946197986602783
Time:  9.347978115081787
Time:  8.685923099517822
Time:  9.228801012039185
Time:  9.846449851989746
Time:  8.72141408920288
Time:  9.236146926879883
Time:  8.854475975036621
Time:  12.44452714920044
Time:  9.913031816482544
Time:  9.850539922714233
Time:  9.923410892486572
Time:  9.35408902168274
Time:  9.44226884841919
Time:  10.03263807296753
Time:  9.409686088562012
Time:  9.204010009765625
Ti

In [185]:
mcts_pl[0][14024079292171647447]

[[765384773236229749,
  {(-1, 0, 1): 'b',
   (-1, 1, 0): 'w',
   (0, -1, 1): '',
   (0, 0, 0): '',
   (0, 1, -1): '',
   (1, -1, 0): '',
   (1, 0, -1): ''},
  1,
  258,
  {765384773236229749: [[3585104155712041760,
     {(-1, 0, 1): 'b',
      (-1, 1, 0): 'w',
      (0, -1, 1): 'b',
      (0, 0, 0): 'w',
      (0, 1, -1): '',
      (1, -1, 0): '',
      (1, 0, -1): ''},
     2,
     1289,
     {3585104155712041760: [[15667646708959168288,
        {(-1, 0, 1): 'b',
         (-1, 1, 0): 'w',
         (0, -1, 1): 'b',
         (0, 0, 0): 'w',
         (0, 1, -1): 'b',
         (1, -1, 0): 'w',
         (1, 0, -1): ''},
        3,
        -999,
        {'l'}],
       [1223680409285247908,
        {(-1, 0, 1): 'b',
         (-1, 1, 0): 'w',
         (0, -1, 1): 'b',
         (0, 0, 0): 'w',
         (0, 1, -1): 'b',
         (1, -1, 0): '',
         (1, 0, -1): 'w'},
        3,
        -999,
        {'l'}],
       [8906344357997116477,
        {(-1, 0, 1): 'b',
         (-1, 1, 0): 'w',
   

In [151]:
def updateTT(path, tt):
    
    gamescore = path[len(path)-1][1]
    
    for i in range(len(path)):
        
        node = path[i][0]
        tt[node][1] += path[i][1]
        
    return tt

In [196]:
def countGameResults(gametree, depth, newdepth, oldkey, statecount, numsamples):
    
    if type(gametree) != list:
#         for key in gametree:
        for i in range(numsamples): # random pick a play
            rnd_key = random.choice(list(gametree))
            nodes = gametree[rnd_key]
            rnd = random.randint(0, len(nodes) - 1)
            node = nodes[rnd]
            
            if depth == newdepth:
                oldkey = rnd_key
            
            if type(node[4]) != set:
                countGameResults(node[4], depth, newdepth+1, oldkey, statecount, 2)
                depth += 1
            else:
                state = list(node[4])[0]
                if oldkey in statecount:
                    if state == 'w':
                        statecount[oldkey][0] += 1
                    elif state == 'l':
                        statecount[oldkey][1] += 1
                    else:
                        statecount[oldkey][2] += 1      
                else:
                    statecount[oldkey] = [1, 1, 1, node[2], node[3]]
                    
    return statecount

gamestate = countGameResults(gametree, 0, 0, 14024079292171647447, {14024079292171647447 : [1, 1, 1]}, 20000)
gamestate

{4826721294065415795: [2, 1, 1, 3, 999],
 6620227438546479916: [1, 1, 2, 3, -499],
 7108538218476928515: [1, 1, 2, 3, -999],
 9678366018592065915: [2, 1, 1, 3, 999],
 14024079292171647447: [33099, 33131, 13765]}

In [270]:
from anytree import Node, RenderTree, search

def createTree(gametree, root):
    
    if type(gametree) != set:
        for key in gametree:
            for node in gametree[key]:
                newnode = Node(node[0], parent=root)
                if type(node[4]) == set:
                    Node(node[2],  parent = newnode)
                    Node(node[3],  parent = newnode)
                    Node(node[4],  parent = newnode)
                createTree(node[4], newnode)
    return root


udo = createTree(gametree, Node(14024079292171647447))

for pre, fill, node in RenderTree(udo):
    print("%s%s        %s%s" % (pre, node.name, node.height, node.depth))

14024079292171647447        40
├── 765384773236229749        31
│   ├── 3585104155712041760        22
│   │   ├── 15667646708959168288        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 1223680409285247908        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 8906344357997116477        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 17920768063577038156        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 6483337940097944406        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   └── 1388885949132615331        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 15663621320990945659        22
│   │   ├── 10014683704764746193        13
│   │   │   ├── 

│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   └── 13861926012348298053        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 16369774792482149714        22
│   │   ├── 8449018650882117499        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 17756034342702738954        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 1779910197049703487        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 13985546648793228758        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 15175858774196803004        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   └── 10021837161276189988        13
│   │       ├── 3        

│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 1421732114750200618        22
│   │   ├── 10014683704764746193        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 4858969165894798677        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 5415128352947122894        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 15764995030942610758        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 7585933188618126757        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   └── 3548142191569913513        13
│   │       ├── 3        04
│   │       ├── 999        04
│   │       └── {'w'}        04
│   ├── 6408835970260141387        22
│   │   ├── 9521187367967734423        13
│   │   │   ├── 3        

│   │       └── {'l'}        04
│   ├── 7156918819712058775        22
│   │   ├── 2134443280045628901        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 17672007824819789758        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 2592252473391669328        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 8824323023439960264        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 11010428261020347642        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   └── 3018449029680961081        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 11040277054520828213        22
│   │   ├── 2134443280045628901        13
│   │   │   ├── 3        04
│   │   │   ├── -999   

│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 9846962586871330424        22
│   │   ├── 17756034342702738954        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 2218760103425125457        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 14415050514508764095        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 10485295804888689447        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 8302928902539583253        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   └── 13985546648793228758        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 8269728831873749722        22
│   │   ├── 17756034342702738954 

...

│   │       └── {'l'}        04
│   ├── 1983726856541769017        22
│   │   ├── 11010428261020347642        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 7109051759999895640        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 10778027689243514820        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 2366770563014211692        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 6865845784709578133        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   └── 1182092385784952479        13
│   │       ├── 3        04
│   │       ├── 999        04
│   │       └── {'w'}        04
│   ├── 16472841709300538327        22
│   │   ├── 16124190792991108738        13
│   │   │   ├── 3        04
│   │   │   ├── -999  

│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 5822132458261992280        22
│   │   ├── 11010428261020347642        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 3018449029680961081        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 11189896583542826114        13
│   │   │   ├── 3        04
│   │   │   ├── -499        04
│   │   │   └── {'d'}        04
│   │   ├── 3314990149964811050        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 1442174229781111796        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   └── 1182092385784952479        13
│   │       ├── 3        04
│   │       ├── 999        04
│   │       └── {'w'}        04
│   ├── 12668777272240604598        22
│   │   ├── 15068449154350554564        13
│   │   │   ├── 3  

│   │   └── 16214469415624393587        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 12297312275585873914        22
│   │   ├── 7109051759999895640        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 3018449029680961081        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 7002804280735268896        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 3314990149964811050        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   ├── 2647120804359482631        13
│   │   │   ├── 3        04
│   │   │   ├── -999        04
│   │   │   └── {'l'}        04
│   │   └── 2366770563014211692        13
│   │       ├── 3        04
│   │       ├── -999        04
│   │       └── {'l'}        04
│   ├── 6175564345512255764  

│   │   │   └── {'w'}        04
│   │   └── 12866989098653142402        13
│   │       ├── 3        04
│   │       ├── 999        04
│   │       └── {'w'}        04
│   ├── 17959563886368241931        22
│   │   ├── 3548142191569913513        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 8824323023439960264        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 3655534262045544145        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 9049087990837757403        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   ├── 8586554931708011510        13
│   │   │   ├── 3        04
│   │   │   ├── 999        04
│   │   │   └── {'w'}        04
│   │   └── 8308456482662332061        13
│   │       ├── 3        04
│   │       ├── 999        04
│   │       └── {'w'}        04
│   

    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   └── 1182092385784952479        13
    │       ├── 3        04
    │       ├── 999        04
    │       └── {'w'}        04
    ├── 17959563886368241931        22
    │   ├── 3548142191569913513        13
    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   ├── 8824323023439960264        13
    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   ├── 3655534262045544145        13
    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   ├── 9049087990837757403        13
    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   ├── 8586554931708011510        13
    │   │   ├── 3        04
    │   │   ├── 999        04
    │   │   └── {'w'}        04
    │   └── 8308456482662332061        13
    │       ├── 3        04
    │    

In [242]:
udo.height, udo.depth #path to the leaf, depth)

(4, 0)

In [288]:
# Node('/14024079292171647447/765384773236229749/3585104155712041760').siblings
# udo.descendants
w = search.findall(udo, filter_=lambda node: node.name in ({'w'}))
l = search.findall(udo, filter_=lambda node: node.name in ({'l'}))
d = search.findall(udo, filter_=lambda node: node.name in ({'w'}, {'l'},{'d'}))
list(d)[0]

Node("/14024079292171647447/765384773236229749/3585104155712041760/15667646708959168288/{'l'}")

In [289]:
for i in udo:
    print(i)

TypeError: 'Node' object is not iterable

In [232]:
# from anytree.exporter import DotExporter
# # graphviz needs to be installed for the next line!
# DotExporter(udo).to_picture("udo.png")

In [152]:
def findTerminal(tempTree, keys, tt):
    path = []
    for key in tempTree: # For every parent
        for node in tempTree[key]: # For every child
            if type(node) == list:
                
                keys += [(key, node[2], node[3])]
                path += [(key, node[3])]
                tt[key] = [node[2], node[3]] #depth, score
                
                if type(node[4]) != set:
                    findTerminal(node[4], keys, tt) 
                else:
                    updateTT(path, tt)
        
    return tt, keys
        
tempTree = mcts_pl[0]
tt, keys = findTerminal(tempTree, [], {})
tt

{3435581961612317: [3, 6993],
 39385405219259975: [3, 6993],
 107821094027263443: [3, 6993],
 170851036519114223: [3, -6993],
 195415070459771228: [3, -1997],
 214368739698587051: [3, 6993],
 271758501592819858: [3, 1],
 299454701419771265: [3, 1],
 448951548086464644: [3, -6993],
 491529979603430455: [3, -1997],
 534088104026184532: [3, 6993],
 603426235887127783: [3, -6993],
 655740937180958221: [2, 1461],
 765384773236229749: [2, 1255],
 821104508467350458: [3, -6993],
 832491755836446473: [3, -1997],
 859074235180505019: [2, 1475],
 873935308560582381: [3, 6993],
 895935174624234010: [3, -1997],
 902897710367561385: [3, -6993],
 924208808177763647: [3, -3995],
 930231086473519168: [3, 1],
 1170259009779012434: [3, 6993],
 1252251588116221657: [2, 1271],
 1255960169098262658: [3, -6993],
 1319523581949337790: [3, 6993],
 1353397388606278372: [3, 6993],
 1421732114750200618: [3, 6993],
 1562037862206324141: [3, 6993],
 1850940307804494616: [2, 1346],
 1917341103570201118: [3, -6993],

In [56]:
import numpy as np

# sort on score, highest first
a = np.array(mcts_pl[0][14024079292171647447])
a[a[:,3].argsort()[::-1]][:, 3]
#     )

array([291, 288, 287, 286, 284, 283, 279, 279, 278, 277, 276, 272, 272,
       269, 269, 269, 268, 267, 264, 263, 262, 260, 260, 259, 259, 259,
       258, 256, 256, 256, 255, 252, 251, 251, 251, 250, 248, 241, 240,
       240, 237, 220], dtype=object)

# Checking Whether the Computer Can Win in One Move

In [ ]:
# Here is the algorithm for our Tic-Tac-Toe AI:
# First, check if we can win in the next move.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, computerLetter, i)
    if isWinner(boardCopy, computerLetter):
        return i

# Checking Whether the Player Can Win in One Move

In [ ]:
# Check if the player could win on their next move and block them.
for i in range(1, 10):
    boardCopy = getBoardCopy(board)
    if isSpaceFree(boardCopy, i):
        makeMove(boardCopy, playerLetter, i)
    if isWinner(boardCopy, playerLetter):
        return i

# Checking Whether the Board Is Full

In [ ]:
def isBoardFull(board):
# Return True if every space on the board has been taken. Otherwise, return False.
for i in range(1, 10):
    if isSpaceFree(board, i):
        return False
    return True

# The Game Loop

In [ ]:
print('Welcome to Omega!')

In [ ]:
# Choosing the Player’s Mark and Who Goes First
playerLetter, computerLetter = inputPlayerLetter()
turn = whoGoesFirst()
print('The ' + turn + ' will go first.')
gameIsPlaying = True

In [ ]:
while gameIsPlaying:
    if turn == 'player':
        # Player's turn
        drawBoard(theBoard)
        move = getPlayerMove(theBoard)
        makeMove(theBoard, playerLetter, move)

    if isWinner(theBoard, playerLetter):
        drawBoard(theBoard)
        print('Hooray! You have won the game!')
        gameIsPlaying = False

    else:
        if isBoardFull(theBoard):
            drawBoard(theBoard)
            print('The game is a tie!')
            break

        else:
            turn = 'computer'

    else:
        # Computer's turn
        move = getComputerMove(theBoard, computerLetter)
        makeMove(theBoard, computerLetter, move)
        if isWinner(theBoard, computerLetter):
            drawBoard(theBoard)
            print('The computer has beaten you! You lose.')
            gameIsPlaying = False
        else:
            if isBoardFull(theBoard):
                drawBoard(theBoard)
                print('The game is a tie!')
                break
            else:
                turn = 'player'

# Asking the Player to Play Again

In [ ]:
print('Do you want to play again? (yes or no)')
if not input().lower().startswith('y'):
    break

In [ ]:
# def printBoard(board):
#     line = ''
#     space = 0
#     for y in range(0, N):
#         for x in range(-N, N*2):
#             try:
#                 line += Hex[(x, y, -x-y)]
#             except:
#                 line += '_'
        
#     return line
# printBoard(Hex)